In [2]:
import pandas as pd
import numpy as np
from glob import glob
import time
from datetime import datetime
import csv
import ccxt

from data_provider import read_data_path, market_to_symbol
from TA_robots import ma_rsi

market = 'ZIL-BTC'
tf = '5m'

start_capital = 0.004
deposit_percent = 0.3

ma_win = 4; rsi_window = 22; rsi_sell = 74

robot = ma_rsi()
robot_parameters = [ma_win, rsi_window, rsi_sell]
robot_name = robot.robot_name + '_' + market + '_' + tf

state_fn = 'Logs/' + robot_name + '.state'
state_file = glob(state_fn)
if state_file == []:
    market_position = 0
    amount  = 0
    buy_sum = 0
else:
    print('State file !!!')
    df_state = pd.read_csv(state_file[-1])    
    market_position = df_state['mp'][0]
    start_capital = df_state['capital'][0]
    amount = df_state['amount'][0]
    buy_sum = df_state['buy_sum'][0]


#Work 30
api_key = "7hOldNclzrzvMe3Xs8otllgdmsKLXIpctjP1eB009P9yd7EzIo5FqunaArrb0YFH"
api_secret = "0J5mmatEvePndM5JwpAIWPYo8ZCDSvxnqxsNPEYXCe35SSF1EDDOaiZQPR0Tenwn"

pd.set_option('precision', 9)
exchange = ccxt.binance({'apiKey': api_key,
                         'secret': api_secret})
commision = 0.00075

In [3]:
def write_trades(df_trades, trade):
    df_state = pd.DataFrame(columns=['mp', 'capital', 'amount'])
    df_state.loc[1] = [market_position, cur_capital, amount]
    df_state.to_csv(state_fn, index = False)
    
    print(trade[1])
    print('{d[0]} {d[1]:10s} {d[2]:.2f} {d[3]:.8f} {d[4]: .8f} {d[5]:.8f} {d[6]:.8f} {d[7]:.8f}'.format(d=trade))
    df_trades.loc[len(df_trades)] = trade
    fn =  'Logs/trades_' + robot_name + '.csv'
    df_trades.to_csv(fn , index = False)    
    

def create_market_buy_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_buy_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol,  order['timestamp'])
    price = trades[0]['price']
    return price

def create_market_sell_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_sell_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol,  order['timestamp'])
    price = trades[0]['price']
    return price
  

In [ ]:
data_path = read_data_path()
df_trades = pd.DataFrame(columns = ['Date', 'Trade type', 'amount', 'close price', 'buy price', 'cumProfit', 'curCapital', 'Deposit'])

symbol = market_to_symbol(market)
cur_capital = start_capital
cum_profit  = 0
deposit = 0
last_time = ''
tf_since_entry = 0

stop= False
print('Start trading:', str(datetime.now()), 'cur_capital:', cur_capital, 'amount:', amount)
while not stop:
    time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
    #pd.read_csv не работает с сash file
    with open(data_path + '/Cash/' + market + '_' + tf + '.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        rl = list(reader)
    if rl != []:
        df = pd.DataFrame(rl[1:])
        df.columns = rl[0]
    df = df.dropna()
    if last_time > df.iloc[-1]['T']:
        print('time error', last_time, df.iloc[-1]['T'])
        continue
    elif last_time < df.iloc[-1]['T']:
        last_time = df.iloc[-1]['T']
        tf_since_entry += 1  
    else:
        continue
    
            
    tf_closes = df["C"].values.astype('float')
    
    price = tf_closes[-1]
    close_time = exchange.iso8601(int(df.iloc[-1]['T'])) 
    signals = robot.signals(tf_closes, *robot_parameters)
    
    #Long entry 
    if  market_position == 0 and signals[-1] == 1:

        amount = cur_capital / price * (1-commision) 
        amount = np.trunc(amount * 100)/100
        buy_price = create_market_buy_order(exchange, symbol, amount)
        
        buy_sum = buy_price * amount + cur_capital * commision
        cur_capital = 0;

        market_position = 1
        tf_since_entry = 0    
 
     
        write_trades(df_trades, [close_time, "entry", amount, price,  buy_price, cum_profit, cur_capital, deposit])
      #Long Exit Profit 
    if market_position == 1 and tf_since_entry > 1 and signals[-1] == -1:
 
        sell_price = create_market_sell_order(exchange, symbol, amount)

        profit = sell_price * amount - buy_sum
        cum_profit += profit
        deposit_sum = max(0, profit) * deposit_percent
        deposit += deposit_sum
        cur_capital = sell_price * amount - deposit_sum
        buy_sum = 0

        market_position = 0

        write_trades(df_trades, [close_time, "exit", amount, price, sell_price , cum_profit, cur_capital, deposit])
    #------------------Post processing---------------------------------------------------


    with open(state_fn, 'w') as state_file:
        state_file.write("time,mp,capital,amount,buy_sum\n") 
        state_file.write("%s,%s,%s,%s, %s" % (close_time, market_position, cur_capital, amount, buy_sum))

    df_stop = pd.read_csv('markets_in_trade.csv')
    stop=df_stop[(df_stop['robot'] == robot.robot_name) & (df_stop['market'] == market) & (df_stop['tf'] == tf)]['stop']
    if stop.shape[0] == 0: 
        print('В файле markets_in_trade.csv нет установок для данного робота')
        break
    else:
        stop = bool(stop.values[0])
    if stop:
        if market_position == 1: 
            sell_price = create_market_sell_order(exchange, symbol, amount)

            profit = sell_price * amount - buy_sum
            cum_profit += profit
            deposit_sum = max(0, profit) * deposit_percent
            deposit += deposit_sum
            cur_capital = sell_price * amount - deposit_sum

            market_position = 0

            write_trades(df_trades, [close_time, "stop", amount, price, sell_price , cum_profit, cur_capital, deposit])
        break

Start trading: 2018-08-25 19:08:36.260562 cur_capital: 0.004 amount: 0
entry
2018-08-25T16:30:00.000Z entry      674.03 0.00000593  0.00000594 0.00000000 0.00000000 0.00000000


'1535164500000'

T    None
O    None
H    None
L    None
C    None
V    None
Name: 500, dtype: object